In [26]:
import requests
from bs4 import BeautifulSoup
import json
import re
import os
import time
import pandas as pd
from datetime import datetime

#Config 

HEADERS = {
    "User-Agent": (
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
        "AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/120.0.0.0 Safari/537.36"
    )
}
MAX_PAGES   = 3
SCRIPT_DIR  = os.getcwd()
OUTPUT_FILE = os.path.join(SCRIPT_DIR, "pittsburgh_events.csv")

def clean(text):
    return " ".join(text.split()) if text else "N/A"

def get_text(el):
    return clean(el.get_text()) if el else "N/A"

In [27]:
#Scrape PGH.events

def scrape_pgh_events():
    pgh_events = []
    for page_num in range(1, MAX_PAGES + 1):
        url = ("https://pgh.events/" if page_num == 1
               else f"https://pgh.events/?page={page_num}")
        print(f"[pgh.events] Fetching page {page_num}: {url}")
        try:
            response = requests.get(url, headers=HEADERS, timeout=15)
            response.raise_for_status()
        except requests.RequestException as e:
            print(f"  ✗ {e}"); break

        soup = BeautifulSoup(response.text, "html.parser")
        day_blocks = soup.select("[class*='day-module--day']")
        if not day_blocks:
            print("  ✗ No day blocks found."); break
        print(f"  ✓ {len(day_blocks)} day block(s) found.")

        for day in day_blocks:
            day_time_el = day.select_one("time")
            day_date = day_time_el.get("datetime", "N/A")[:10] if day_time_el else "N/A"

            for card in day.select("[class*='event-module--event']"):
                name_el    = card.select_one("[class*='event-module--mainLink']")
                event_name = get_text(name_el)

                link_el    = name_el if (name_el and name_el.name == "a") else card.select_one("a[href]")
                source_url = link_el["href"] if link_el else "N/A"
                if source_url != "N/A" and source_url.startswith("/"):
                    source_url = "https://pgh.events" + source_url

                location = "N/A"
                for p in card.select("p"):
                    if not p.get("class"):
                        txt = clean(p.get_text())
                        if txt and txt != "N/A":
                            location = txt; break

                card_time_el = card.select_one("time")
                event_date   = day_date
                event_time   = "N/A"
                if card_time_el:
                    raw_dt = card_time_el.get("datetime", "")
                    if raw_dt and "T" in raw_dt:
                        try:
                            dt = datetime.strptime(re.sub(r'[+-]\d{4}$', '', raw_dt), "%Y-%m-%dT%H:%M:%S")
                            event_date = dt.strftime("%Y-%m-%d")
                            event_time = dt.strftime("%I:%M %p")
                        except ValueError:
                            event_date = raw_dt[:10]

                price_el = card.select_one("[class*='price']") or card.select_one("[class*='cost']")
                price    = get_text(price_el)
                if price == "N/A":
                    m = re.search(r'(Free|\$[\d,.]+)', card.get_text(), re.IGNORECASE)
                    price = m.group(0) if m else "N/A"

                pgh_events.append({
                    "event_name": event_name, "date": event_date,
                    "time": event_time,       "location": location,
                    "price": price,           "source": "pgh.events",
                    "url": source_url,
                })

        print(f"  → {len(pgh_events)} events so far.")
        time.sleep(1.5)

    print(f"\n[pgh.events] Total: {len(pgh_events)} events\n")
    return pgh_events


In [28]:
#Eventbrite Pre Scrape

def parse_eventbrite_datetime(soup, raw_html):
    # Strategy 1: <time datetime="...">
    time_el = soup.select_one("time[datetime]")
    if time_el:
        try:
            dt = datetime.fromisoformat(time_el.get("datetime", "").replace("Z", "+00:00"))
            return dt.strftime("%Y-%m-%d"), dt.strftime("%I:%M %p")
        except ValueError: pass
    # Strategy 2: JSON-LD structured data
    for script in soup.select("script[type='application/ld+json']"):
        try:
            data  = json.loads(script.string or "")
            start = data.get("startDate", "")
            if start:
                dt = datetime.fromisoformat(start.replace("Z", "+00:00"))
                return dt.strftime("%Y-%m-%d"), dt.strftime("%I:%M %p")
        except: continue
    # Strategy 3: regex on raw HTML
    iso = re.search(r'"startDate"\s*:\s*"(\d{4}-\d{2}-\d{2}T\d{2}:\d{2}:\d{2})', raw_html)
    if iso:
        try:
            dt = datetime.fromisoformat(iso.group(1))
            return dt.strftime("%Y-%m-%d"), dt.strftime("%I:%M %p")
        except: pass
    # Strategy 4: human-readable text 
    date_pat = re.compile(
        r'(?:Monday|Tuesday|Wednesday|Thursday|Friday|Saturday|Sunday),?\s*'
        r'(January|February|March|April|May|June|July|August|September|October|November|December)'
        r'\s+(\d{1,2})(?:,?\s*(\d{4}))?', re.IGNORECASE)
    time_pat = re.compile(r'\b(\d{1,2}:\d{2}\s*(?:AM|PM))\b', re.IGNORECASE)
    text = soup.get_text(" ")
    event_date = event_time = "N/A"
    dm = date_pat.search(text)
    tm = time_pat.search(text)
    if dm:
        try:
            dt = datetime.strptime(f"{dm.group(1)} {dm.group(2)} {dm.group(3) or '2026'}", "%B %d %Y")
            event_date = dt.strftime("%Y-%m-%d")
        except: event_date = f"{dm.group(1)} {dm.group(2)}, 2026"
    if tm: event_time = tm.group(1).upper().replace(" ", "")
    return event_date, event_time

def parse_eventbrite_location(soup):
    for sel in ["[data-spec='venue-name']", "[class*='venue-name']",
                "[class*='location-info__address']", "address"]:
        el = soup.select_one(sel)
        if el:
            txt = clean(el.get_text())
            if txt and len(txt) < 100: return txt
    candidates = [clean(el.get_text()) for el in soup.find_all(["p","span","div","address"])
                  if "Pittsburgh" in clean(el.get_text()) and 5 < len(clean(el.get_text())) < 80]
    return min(candidates, key=len) if candidates else "N/A"

In [29]:
#Scrape Eventbrite 

def scrape_eventbrite():
    # Step 1: collect event URLs from listing pages
    print("[Eventbrite] Step 1: Collecting event URLs...")
    eb_urls = []
    for page_num in range(1, MAX_PAGES + 1):
        url = ("https://www.eventbrite.com/d/pa--pittsburgh/all-events/"
               if page_num == 1
               else f"https://www.eventbrite.com/d/pa--pittsburgh/all-events/?page={page_num}")
        print(f"  Fetching listing page {page_num}")
        try:
            response = requests.get(url, headers=HEADERS, timeout=15)
            response.raise_for_status()
        except requests.RequestException as e:
            print(f"  ✗ {e}"); break
        soup  = BeautifulSoup(response.text, "html.parser")
        found = []
        for a in soup.select("a[href*='/e/']"):
            href = a["href"].split("?")[0]
            if href not in eb_urls and href not in found: found.append(href)
        eb_urls.extend(found)
        print(f"  ✓ {len(found)} URLs found on page {page_num}.")
        time.sleep(1.5)

    # Step 2: fetch each event detail page
    print(f"\n[Eventbrite] {len(eb_urls)} URLs. Fetching detail pages...\n")
    eb_events = []
    for i, event_url in enumerate(eb_urls):
        print(f"  [{i+1}/{len(eb_urls)}] {event_url}")
        try:
            resp = requests.get(event_url, headers=HEADERS, timeout=15)
            resp.raise_for_status()
        except: time.sleep(1); continue
        detail     = BeautifulSoup(resp.text, "html.parser")
        name_el    = detail.select_one("h1") or detail.select_one("[class*='event-title']")
        event_name = get_text(name_el)
        event_date, event_time = parse_eventbrite_datetime(detail, resp.text)
        location               = parse_eventbrite_location(detail)
        price_el = detail.select_one("[class*='ticket-price']") or detail.select_one("[class*='conversion-bar']")
        price    = get_text(price_el)
        if price == "N/A":
            m = re.search(r'(Free|\$[\d,.]+)', resp.text)
            price = m.group(0).capitalize() if m else "N/A"
        eb_events.append({
            "event_name": event_name, "date": event_date,
            "time": event_time,       "location": location,
            "price": price,           "source": "Eventbrite",
            "url": event_url,
        })
        time.sleep(1.2)

    print(f"\n[Eventbrite] Total: {len(eb_events)} events\n")
    return eb_events

In [30]:
#Clean Data

MANUAL_LOCATION_FIXES = {
    "Eddy TheatreWoodland":         "Eddy Theatre",
    "Wyndham Grand":                "Wyndham Grand Pittsburgh Downtown",
    "The Circuit Center Hot Metal": "The Circuit Center",
    "1139 Penn":                    "1139 Penn Ave",
}

def clean_location(loc):
    if not isinstance(loc, str) or loc == "N/A": return loc
    if not re.match(r'^\d', loc):
        loc = re.sub(r'([a-zA-Z])(\d)', r'\1', loc).strip()
    loc = re.split(r'\s+\d{1,5}\s+', loc)[0].strip()
    loc = re.sub(r',?\s*Pittsburgh.*$', '', loc, flags=re.IGNORECASE).strip()
    loc = re.sub(r'\s+(Road|Street|Ave|Avenue|Blvd|Boulevard|Drive|Lane|Way)$',
                 '', loc, flags=re.IGNORECASE).strip()
    return loc.strip(" ,") if loc else "N/A"

def build_dataframe(all_events):
    df = pd.DataFrame(all_events, columns=[
        "event_name", "date", "time", "location", "price", "source", "url"
    ])
    df = df[df["event_name"].str.strip().str.len() > 0]
    df = df[df["event_name"] != "N/A"]
    df.drop_duplicates(subset=["event_name", "date"], inplace=True)
    df.reset_index(drop=True, inplace=True)
    return df

def clean_dataframe(df):
    df = df.fillna("N/A")
    df["location"] = df["location"].apply(clean_location)
    df["location"] = df["location"].replace(MANUAL_LOCATION_FIXES)
    df["price"]    = df["price"].apply(lambda p: p.rstrip(".") if isinstance(p, str) else p)
    return df

def save_csv(df, path):
    df.to_csv(path, index=False, encoding="utf-8-sig")
    print(f"\n{'='*50}")
    print(f"✅  {len(df)} events saved to {path}")
    print(f"{'='*50}")
    print(df[["event_name","date","time","location","price","source"]].to_string(index=False))

def load_csv(path):
    df = pd.read_csv(path)
    return df.fillna("N/A")


In [31]:
#Output Main

def prompt_user():
    cached_exists = os.path.exists(OUTPUT_FILE)
    print("=" * 60)
    print("  Pittsburgh Date Night App — Lime Team")
    print("=" * 60)
    if cached_exists:
        print(f"\n  Cached dataset found: {OUTPUT_FILE}\n")
        print("  [1] Use cached data  (instant)")
        print("  [2] Download fresh data  ( ~3-5 minutes)\n")
        while True:
            choice = input("  Enter 1 or 2: ").strip()
            if choice == "1": return False
            elif choice == "2":
                confirm = input("  Are you sure? (y/n): ").strip().lower()
                return confirm == "y"
            else: print("  Please enter 1 or 2.")
    else:
        print("\n  No cached data found. Fresh download required (~3-5 mins).")
        input("  Press Enter to start...")
        return True

def main():
    use_fresh = prompt_user()
    if use_fresh:
        print("\n[Starting fresh scrape...]\n")
        all_events = scrape_pgh_events() + scrape_eventbrite()
        if not all_events:
            print("No events collected."); return
        df = build_dataframe(all_events)
        df = clean_dataframe(df)
    else:
        print(f"\n[Loading cached data...]\n")
        df = load_csv(OUTPUT_FILE)
        df = clean_dataframe(df)
    save_csv(df, OUTPUT_FILE)

main()


  Pittsburgh Date Night App — Lime Team

  No cached data found. Fresh download required (~3-5 mins).

[Starting fresh scrape...]

[pgh.events] Fetching page 1: https://pgh.events/
  ✓ 1 day block(s) found.
  → 11 events so far.
[pgh.events] Fetching page 2: https://pgh.events/?page=2
  ✓ 1 day block(s) found.
  → 22 events so far.
[pgh.events] Fetching page 3: https://pgh.events/?page=3
  ✓ 1 day block(s) found.
  → 33 events so far.

[pgh.events] Total: 33 events

[Eventbrite] Step 1: Collecting event URLs...
  Fetching listing page 1
  ✓ 20 URLs found on page 1.
  Fetching listing page 2
  ✓ 20 URLs found on page 2.
  Fetching listing page 3
  ✓ 20 URLs found on page 3.

[Eventbrite] 60 URLs. Fetching detail pages...

  [1/60] https://www.eventbrite.com/e/robotics-tech-happy-hour-tickets-1968235596653
  [2/60] https://www.eventbrite.com/e/a-conversation-with-margaret-hoover-and-jennifer-palmieri-tickets-1976939690833
  [3/60] https://www.eventbrite.com/e/mamma-mia-tickets-1980133376